## Maximum Likelihood Estimator(MLE)   

A MLE coincides with the most probable Bayesian Estimator given a uniform prior distribution on the parameters. Indeed, the maximum a posteriori estimate is the parameter $\Theta$ that maximize the probability of $\Theta$ given tha data, given by Bayes's theorem:
$$
    P(\Theta \mid x_1, \dots, x_n) = \frac{f(x_1, \dots, x_n \mid \Theta) \ P(\Theta)}{P(x_1, \dots, x_n)}
$$

Therefore, $\textbf{for uniform prior distribution $P(\Theta)$}$, the bayesian estimator coincides with the MLE.


In [10]:
# %%
from tqdm import tqdm
import torch
import torch.optim as optim

batch_size = 10
n_epochs = 1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Sample data: Assume it is generated from 10 normal distributions, each with different mean and std
torch.manual_seed(0)
means_true = torch.linspace(1, 10, batch_size, device=device)  # True means for each distribution
stds_true = torch.linspace(1, 2, batch_size, device=device)  # True stds for each distribution
data = torch.stack([torch.normal(mean, std, size=(10000,), device=device) for mean, std in zip(means_true, stds_true)])  # shape: (10, 10000)

# Parameters to estimate: Mean (mu) and standard deviation (sigma) for each of the 10 distributions
mu = torch.randn(batch_size, requires_grad=True, device=device)  # Initialize mu for 10 distributions
sigma = torch.randn(batch_size, requires_grad=True, device=device)  # Initialize sigma for 10 distributions

# Optimizer
optimizer = optim.Adam([mu, sigma], lr=0.1)

# Maximum Likelihood Estimation using Gaussian log-likelihood
progress_bar = tqdm(range(n_epochs), desc="Training Progress")
for step in progress_bar:
    optimizer.zero_grad()

    # Convert log(sigma) to sigma to ensure sigma is positive
    sigma_clamped = torch.exp(sigma)
    
    # Negative log-likelihood of the Gaussian distribution
    nll = -torch.mean(-0.5 * torch.log(2 * torch.pi * sigma_clamped**2).unsqueeze(1) 
                      - 0.5 * ((data - mu.unsqueeze(1))**2 / (sigma_clamped.unsqueeze(1)**2)))

    # Backpropagate and optimize
    nll.backward()
    optimizer.step()
        
    # Calculate RMSE between estimated parameters and true values
    rmse_mu = torch.sqrt(torch.mean((mu - means_true) ** 2)).item()
    rmse_sigma = torch.sqrt(torch.mean((sigma_clamped - stds_true) ** 2)).item()

    # Update the tqdm description with current estimates for every step
    progress_bar.set_description(
        f"Step {step+1} - NLL: {nll.item():.4f}, RMSE_mu: {rmse_mu:.4f}, RMSE_sigma: {rmse_sigma:.4f}"
    )

print("\n")
# Final estimated parameters
print(f'Estimated means: {mu}')
print(f'Estimated standard deviations: {sigma_clamped}')
print(f'RMSE for mu: {rmse_mu:.4f}')
print(f'RMSE for sigma: {rmse_sigma:.4f}')

Step 1000 - NLL: 1.8015, RMSE_mu: 0.0212, RMSE_sigma: 0.0057: 100%|██████████| 1000/1000 [00:03<00:00, 297.72it/s]




Estimated means: tensor([ 0.9921,  2.0002,  3.0183,  3.9970,  5.0182,  6.0056,  6.9943,  7.9591,
         9.0311, 10.0327], device='cuda:0', requires_grad=True)
Estimated standard deviations: tensor([1.0002, 1.1110, 1.2147, 1.3455, 1.4435, 1.5568, 1.6692, 1.7804, 1.8970,
        1.9940], device='cuda:0', grad_fn=<ExpBackward0>)
RMSE for mu: 0.0212
RMSE for sigma: 0.0057


Mean shape: torch.Size([10])
Covariance matrix shape: torch.Size([10, 10])
